import pandas as pd
import numpy as np
import math
import os
import warnings
print('Modulos importados')# Algoritmos auxiliares

En este script se plantea mostrar cómo funcionan algortimos encontrados en internet. Esto, debido a que hay veces en las que son más eficientes que los algoritmos propios, por lo que solo pondré los que son más eficientes que los propios y que haya encontrado.

In [1]:
import pandas as pd
import numpy as np
import math
import os
import warnings
print('Modulos importados')

Modulos importados


In [2]:
# Directorio de trabajo
print("Directorio de trabajo previo: " + str(os.getcwd()))
# Cambiemoslo
os.chdir('/home/usuario/Documentos/Github/Proyectos/MLB_HN/')

Directorio de trabajo previo: /home/usuario/Documentos/Github/Proyectos/MLB_HN/Programming_Test


### Valores únicos por fila

Obtengamos los valores únicos por fila a través de distintos métodos

In [3]:
lst = [[12, 12, 25], [34, 3, 30],
       [26, 26, 26], [np.nan, 1, 22], 
       [np.nan, np.nan, np.nan]]

In [4]:
df = pd.DataFrame(lst, columns =['FName', 'LName', 'Age'], dtype = float)
df

,FName,LName,Age
0,12.0,12.0,25.0
1,34.0,3.0,30.0
2,26.0,26.0,26.0
3,NaN,1.0,22.0
4,NaN,NaN,NaN


In [5]:
list(map(set,df.values))

[{12.0, 25.0}, {3.0, 30.0, 34.0}, {26.0}, {nan, 1.0, 22.0}, {nan, nan, nan}]

In [6]:
df.stack().groupby(level=0).apply(lambda x: x.unique().tolist())

0         [12.0, 25.0]
1    [34.0, 3.0, 30.0]
2               [26.0]
3          [1.0, 22.0]
dtype: object

El anterior es el mejor método

In [7]:
df.T.agg([pd.unique])

,0,1,2,3,4
unique,"[12.0, 25.0]","[34.0, 3.0, 30.0]",[26.0],"[nan, 1.0, 22.0]",[nan]


### Contar valores únicos por fila

In [8]:
df = pd.DataFrame(lst, columns =['FName', 'LName', 'Age'], dtype = float)

df['num_uniq_1'] = [len(set(v[pd.notna(v)].tolist())) for v in df.values]
df

,FName,LName,Age,num_uniq_1
0,12.0,12.0,25.0,2
1,34.0,3.0,30.0,3
2,26.0,26.0,26.0,1
3,NaN,1.0,22.0,2
4,NaN,NaN,NaN,0


In [9]:
df = pd.DataFrame(lst, columns =['FName', 'LName', 'Age'], dtype = float)

# df.join(df.stack().groupby(level=0).nunique().to_frame('num_uniq'))
df['num_uniq_2'] = df.stack().groupby(level=0).nunique()
df

,FName,LName,Age,num_uniq_2
0,12.0,12.0,25.0,2.0
1,34.0,3.0,30.0,3.0
2,26.0,26.0,26.0,1.0
3,NaN,1.0,22.0,2.0
4,NaN,NaN,NaN,NaN


In [10]:
df = pd.DataFrame(lst, columns =['FName', 'LName', 'Age'], dtype = float)

df['num_uniq_3'] = df.apply(pd.Series.nunique, axis=1)
df

,FName,LName,Age,num_uniq_3
0,12.0,12.0,25.0,2
1,34.0,3.0,30.0,3
2,26.0,26.0,26.0,1
3,NaN,1.0,22.0,2
4,NaN,NaN,NaN,0


### Buscar un valor entre dos dataframes

Nos interesa hallar la fila en la que dos dataframes tienen un mismo valor con respecto a una columna

In [11]:
df1 = pd.DataFrame({'Time': ['8a', '10p'], 'V1': [1, 2], 'V2': [3, 4]})
df2 = pd.DataFrame({'fn': ['8.txt', '10.txt'], 'V1': [3, 2], 'V2': [3, 4]})

Primera solución

In [12]:
df1.merge(df2, on=['V1', 'V2'], how='outer').dropna()

,Time,V1,V2,fn
1,10p,2,4,10.txt


In [13]:
df2.merge(df1,how="inner",on=["V1","V2"])[["fn","Time"]]

,fn,Time
0,10.txt,10p


In [14]:
pd.concat([df1, df2.reindex(df1.index)], axis=1)

,Time,V1,V2,fn,V1,V2
0,8a,1,3,8.txt,3,3
1,10p,2,4,10.txt,2,4


Segunda solución y óptima para los propósitos

In [15]:
for row in range(len(df1)):
    if (df1.iloc[row,1:] == df2.iloc[row,1:]).all() == True: 
        print(df1.iloc[row], df2.iloc[row])

Time    10p
V1        2
V2        4
Name: 1, dtype: object fn    10.txt
V1         2
V2         4
Name: 1, dtype: object


In [16]:
client = boto3.client('s3')

obj = client.get_object(Bucket='', Key='')
data = obj['Body'].read()
df1 = pd.read_excel(io.BytesIO(data), sheet_name='0')
df2 = pd.read_excel(io.BytesIO(data), sheet_name='1')

head = df2.columns[0]
print(head)

data = df1.iloc[[8],[0]].values[0]
print(data)

print(df2)
df2.columns = df2.iloc[0]
df2 = df2.drop(labels=0, axis=0)
df2['Head'] = head
df2['ID'] = pd.Series([data,data])

print(df2)
df2.to_csv('test.csv',index=False)

NameError: name 'boto3' is not defined

## Merge de bases de datos de forma condicional

Lo que se hará es crear un algoritmo que filtre las bases de datos cuyos años de contrato restante sean mayores a uno y luego mostrarlos en bucle hasta que dicha cantidad sea cero

In [ ]:
# Auxiliares:
free_agents = 'Data/Free_Agents/Free_Agents_'
hitting = 'Data/Statistics/Hitting/hitting_'
pitching = 'Data/Statistics/Pitching/pitching_'
salary = 'Data/Salary/Salary_'
teams = 'ETL_data/Teams/free_agents_team_'
csv = '.csv'
period = 11
# Originales:
df_free_agents = [None]*period
df_hitting = [None]*period
df_pitching = [None]*period
df_salary = [None]*period
df_teams = [None]*period
# Copias:
df_free_agents_copy = [None]*period
df_hitting_copy = [None]*period
df_pitching_copy = [None]*period
df_salary_copy = [None]*period
df_teams_copy = [None]*period
# Producto final:
df_pitchers = [None]*period
df_hitters = [None]*period
df_pitchers_free_agents = [None]*period
df_hitters_free_agents = [None]*period
df_pitchers_no_free_agents = [None]*period
df_hitters_no_free_agents = [None]*period

In [ ]:
for i in range(0,period):    
    df_free_agents[i] = pd.read_csv(free_agents + str(2011 + i) + csv)
    df_hitting[i] = pd.read_csv(hitting + str(2011 + i) + csv)
    df_pitching[i] = pd.read_csv(pitching + str(2011 + i) + csv)
    df_salary[i] = pd.read_csv(salary + str(2011 + i) + csv)
    df_teams[i] = pd.read_csv(teams + str(2011 + i) + csv)
    
    df_free_agents_copy[i] = df_free_agents[i].copy()
    df_hitting_copy[i] = df_hitting[i].copy()
    df_pitching_copy[i] = df_pitching[i].copy()
    df_salary_copy[i] = df_salary[i].copy()
    df_teams_copy[i] = pd.read_csv(teams + str(2011 + i) + csv)

Veamos el contenido de las bases de datos de los agentes libres

In [ ]:
df_free_agents_copy[4].head()

Filtremos aquellos que tienen más de un año de contrato

In [ ]:
df_free_agents_copy[4][df_free_agents_copy[4]['YRS'] > 1]

Ahora, haremos un bucle para mostrar progrsivamente los jugadores con contratos que duren más y más años.

In [ ]:
period = 11

Veamos la cantidad de datos que se añadirían a cada base de datos anual con respecto a la base de datos de algún año previo

In [ ]:
for year in range(0,period):
    
    max_year_contract = max(df_free_agents_copy[year]['YRS'])
    print("Max year = " + str(max_year_contract))
    print('Year = ' + str(2011 + year))
    
    for incremento in range(0,max_year_contract - 1):
        merge = df_free_agents_copy[year][df_free_agents_copy[year]['YRS'] > 1 + incremento]
        
        print("Año base de datos = " + str(2011 + 1 + incremento))
        print("Len = " +str(len(merge)))
    print("\n")
    print("\n")

Algo que notamos es que hay contratos que se extienden a periodos fuera del año máximo de análisis, 2021. Debido a esto, tendremos que restringir el año a indexar.

In [ ]:
for year in range(0,period):
    
    max_year_contract = max(df_free_agents_copy[year]['YRS'])
    print("Max year = " + str(max_year_contract))
    print('Year = ' + str(2011 + year))
    
    for incremento in range(0,max_year_contract - 1):
        diff_t = 1 + incremento
        real_year = 2011 + diff_t
        year_bound = 2021

        if real_year <= year_bound:
            merge = df_free_agents_copy[year][df_free_agents_copy[year]['YRS'] > diff_t]

            print("Año base de datos = " + str(2011 + diff_t))
            print("Len = " +str(len(merge)))
    print("\n")
    print("\n")

Un problema que se tiene que tomar en cuenta es que si añadimos los datos a bases de datos posteriores a un año, estas se añadirán otra vez. Debido a esto, es más sencillo controlar esto si guardamos los cambios debido a un dataframe en distintos dataframes.

In [ ]:
period_t = period - 1
df_contracts = [None]*(period_t)

In [ ]:
len(df_contracts)

Solo serán 10 años los que tenemos que tomar en cuenta puesto que los contratos de la base de datos del 2021 no se añadirán a ninguna otra base de datos. Por otro lado, como en el año 2011 no hay ninguna contribución para contratos futuros, omitiremos ese año

In [ ]:
for years in range(1,period_t):
    diff_remain = period_t - years
    df_contracts[years] = [None]*diff_remain
    
    print("Year = " + str(2011 + years))
    print("Length = " + str(len(df_contracts[years])))
    for sub_year in range(0,len(df_contracts[years])):
        print(df_contracts[years][sub_year])
    
    print("")

Guardemos en cada una de las entradas correspondientes los distintos registros de jugadores con contratos para más de una temporada.

In [ ]:
for year in range(1,period_t):
    
    max_year_contract = max(df_free_agents_copy[year]['YRS'])
    years = max_year_contract - 1
    df_contracts[year] = [None]*years
    
    print("Max year = " + str(max_year_contract))
    print('Year = ' + str(2011 + year))
    
    for incremento in range(0,years):
        diff_t = 1 + incremento
        real_year = 2011 + year + diff_t
        year_bound = 2021

        if real_year <= year_bound:
            df_contracts[year][incremento] = df_free_agents_copy[year][df_free_agents_copy[year]['YRS'] > diff_t]
            
            print("Año base de datos = " + str(real_year))
    print("\n")
    print("\n")

Veamos algunos ejemplos

In [ ]:
df_contracts[1][2]

In [ ]:
df_contracts[3][5]

Concatenemos estas bases de datos con las de los agentes libres

In [ ]:
for year in range(1,period_t):
    
    years = len(df_contracts[year])
    
    print("Max year = " + str(max_year_contract))
    print('Year = ' + str(2011 + year))
    
    for incremento in range(0,years):
        diff_t = 1 + incremento
        real_year = 2011 + year + diff_t
        year_bound = 2021

        if real_year <= year_bound:
            frames = [df_free_agents_copy[year + diff_t], df_contracts[year][incremento]]
            
            df_free_agents_copy[year + diff_t] = pd.concat(frames)

            df_free_agents_copy[year + diff_t].reset_index(drop = True, inplace = True)
            df_free_agents_copy[year + diff_t] = df_free_agents_copy[year + diff_t].sort_values(by = 'Rank', ascending = True)
            
            print("Año base de datos = " + str(real_year))
    print("\n")
    print("\n")

Verifiquemos el contenidos de las bases de datos de los agentes

In [ ]:
for year in range(0,period):
    print(df_free_agents_copy[year].shape)

In [ ]:
for year in range(0,period):
    print(df_free_agents_copy[year].loc[0:5])